# Setup

In [1]:
import os
import sys

PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
PYTHON_CODE_ROOT = os.path.join(PROJECT_ROOT, 'system', 'python')
sys.path.append(PYTHON_CODE_ROOT)
%load_ext autoreload
%autoreload 2

In [4]:
from aieng.common.settings import gs

# Working with Data
PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [7]:
import torch
from torch.utils.data import DataLoader
import torchvision as tv
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [8]:
training_data = tv.datasets.FashionMNIST(root=gs.DATA_ROOT, train=True, download=True, transform=ToTensor())
test_data = tv.datasets.FashionMNIST(root=gs.DATA_ROOT, train=False, download=True, transform=ToTensor())


Extracting /home/host/space/main/projects/system/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/host/space/main/projects/system/data/FashionMNIST/raw




Extracting /home/host/space/main/projects/system/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/host/space/main/projects/system/data/FashionMNIST/raw




Extracting /home/host/space/main/projects/system/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/host/space/main/projects/system/data/FashionMNIST/raw




Extracting /home/host/space/main/projects/system/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/host/space/main/projects/system/data/FashionMNIST/raw

Processing...
Done!


/home/host/dev/miniconda/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554793803/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


# Creating Models

To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [13]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28*28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,10),
            torch.nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Optimizing the model parameters
To train a model, we need a loss function and an optimizer.

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [16]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg_loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n ---------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
 ---------------
loss: 2.306196 [    0/60000]
loss: 2.301225 [ 6400/60000]
loss: 2.293594 [12800/60000]
loss: 2.286880 [19200/60000]
loss: 2.289195 [25600/60000]
loss: 2.293632 [32000/60000]
loss: 2.283541 [38400/60000]
loss: 2.279965 [44800/60000]
loss: 2.277376 [51200/60000]
loss: 2.283684 [57600/60000]
Test Error: 
 Accuracy: 23.2%, Avg_loss: 0.035556 

Epoch 2
 ---------------
loss: 2.277967 [    0/60000]
loss: 2.263395 [ 6400/60000]
loss: 2.238126 [12800/60000]
loss: 2.230622 [19200/60000]
loss: 2.254377 [25600/60000]
loss: 2.261183 [32000/60000]
loss: 2.240939 [38400/60000]
loss: 2.232560 [44800/60000]
loss: 2.218651 [51200/60000]
loss: 2.238461 [57600/60000]
Test Error: 
 Accuracy: 24.4%, Avg_loss: 0.034432 

Epoch 3
 ---------------
loss: 2.214627 [    0/60000]
loss: 2.180773 [ 6400/60000]
loss: 2.134492 [12800/60000]
loss: 2.127547 [19200/60000]
loss: 2.130885 [25600/60000]
loss: 2.167366 [32000/60000]
loss: 2.107162 [38400/60000]
loss: 2.096127 [44800/60000]
loss: 2.0

# Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [18]:
model_root = gs.MODELS_ROOT + "/pytorch_quickstart"
if not os.path.exists(model_root):
    os.makedirs(model_root)
torch.save(model.state_dict(), f"{model_root}/mlp.pth")

# Loading Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [19]:
model = NeuralNetwork()
model.load_state_dict(torch.load( f"{model_root}/mlp.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [21]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
